In [48]:
import pandas as pd
import numpy as np
import xlrd
from datetime import datetime

hum = ['HIS 315K', 'HIS 315L','RHE 306', 'RHE 309K', 'AET 304', 'NSC 309', 'CS 302']
courseList = {"CS 302": "Computer Science",
        "RHE 306": "ENGL 1301",
        "RHE 309K": "ENGL 1302",
        "GEO 302E": "Geoscience",
        "HIS 315K": "HIST 1301",
        "HIS 315L": "HIST 1302",
        "M 305G": "Precalculus",
        "PHY 302K": "PHYS 1301",
        "PHY 302L": "PHYS 1302",
        "PHY 1403": "PHY 1403",
        "SDS 302": "Statistics",
        "AET 304": "AET",
        "CH 301": "Chemistry",
        "CH 104M": "Chemistry Lab",
        "M 301": "College Algebra",
        "NSC 309": "NSC 309"}
courseList = pd.Series(courseList)

def remove(data):
    removal = []
    for i in removal:
        data = data[data["Course Code"] != i]
    return data
    
# Naming convention to be updated
def standardize(data):
    convention = courseList
    for i in convention.index:
        data.loc[data["Course Code"] == i, "Course Title"] = convention[i]
    return data

#Load Data
def get_master(master, district = ''):
    data = pd.read_csv(master)
    data = remove(data)
    data = data[data["Eligibility Status Final Simple"] != 'HS Only']
    data = standardize(data)
    if district == '':
        return data
    return data[data["District Name"] == district]

def read_data(master, district):
    data = get_master(master, district)
    UT = data[data["Partner"] == "UT Austin OnRamps"]
    TTU = data[data["Partner"] == "TTU OnRamps"]
    return UT, TTU

def read_data_without_differentiate(master, district):
    data = get_master(master, district)
    return data

# Get Row totals
def get_row_total(data2, clean = True):
    if clean:
        data = pd.DataFrame(columns = data2.columns)
        data['Total'] = 0
        for index, row in data2.iterrows():
            rowSum = sum(data2.loc[index])
            if rowSum > 0:
                data.loc[index] = data2.loc[index]
                data.loc[index, 'Total'] = rowSum
    else:
        data = data2   
        data['Total'] = 0
        for index, row in data.iterrows():
            data.loc[index, 'Total'] = sum(data.loc[index])
    return data

# Get Column totals
def get_column_total(data2, clean = True):
    if clean:
        data = pd.DataFrame(index = data2.index)
        for i in data2.columns:
            colSum = data2[i].sum()
            if colSum > 0:
                data[i] = data2[i]
                data.loc['Total', i] = colSum
    else:
        data = data2
        data.loc['Total'] = 0
        for i in data.columns:
            data.loc['Total', i] = data[i].sum()
    return data
    

def get_unique_values(date, label):
    return data[label].drop_duplicates()

def get_Census(data, district, census = True, humanity = False, course = None, campus = None):
    data = data[data["District Name"] == district]
    if census == False:
        data = data[data["Withdraw Status"] == "No"]
    else:
        if humanity:
            course = courseList[hum]
        else:
            course = courseList[~courseList.index.isin(hum)]
        campus = data["Campus Name"].drop_duplicates().sort_values()
    ans = pd.DataFrame(index = pd.Index(course), columns = campus)
    for i in campus:
        ans[i] = data[data["Campus Name"] == i]["Course Title"].value_counts()
    ans = ans.sort_index()
    ans = ans.fillna(0)
    ans = get_row_total(ans, census)
    ans = get_column_total(ans, census)
    return ans

def fill_row(row, num, fill = ''):
    while len(row) < num:
        row.append(fill)
    return row

def Figure_1_4_a_QC(data, district_name):
    census = get_Census(data, district_name)
    #print(census)
    endOfTerm = get_Census(data, district_name, census = False, course = census.index, campus = census.columns)
    #print(endOfTerm)
    indexRow = ['', '', '', 'Course Code', 'Enrollment Dates', None]
    if not census.columns.tolist():
        template = [['', '', '', ''],
            ['', f'Figure 1.4 shows the number of students by campus who were enrolled in each OnRamps distance college course at census* and at the end of the academic term. Students could enroll in multiple course.\n\nThis figure is divided according to course subject. Figure 1.4a includes math and science courses and Figure 1.4b includes humanities, arts, and technology courses. Figures are only displayed if the course(s) are offered at {district_name}.\n*Census is the date on which OnRamps confirms students for reporting purposes only.\n\nFigure 1.4a: Student enrollment in math and science courses at census & end of year at campus level in the 2018-2019 academic year', '', ''],
        ]
        return template
    
    # census.columns.tolist() = a list of campus names
    indexRow = indexRow + census.columns.tolist() + [None]
    
    template = [fill_row([], len(indexRow)),
        fill_row(['', f'Figure 1.4 shows the number of students by campus who were enrolled in each OnRamps distance college course at census* and at the end of the academic term. Students could enroll in multiple course.\n\nThis figure is divided according to course subject. Figure 1.4a includes math and science courses and Figure 1.4b includes humanities, arts, and technology courses. Figures are only displayed if the course(s) are offered at {district_name}.\n*Census is the date on which OnRamps confirms students for reporting purposes only.\n\nFigure 1.4a: Student enrollment in math and science courses at census & end of year at campus level in the 2018-2019 academic year'], len(indexRow)),
        fill_row([], len(indexRow)),
        indexRow
        ]
    for i in census.index:
        censusTag = 'At Census'
        finalTag = 'End of Academic Term'
        if i == 'Total':
            censusTag = 'Census Enrollment'
            finalTag = 'Final Enrollment'
        lrow = ['', '', '', i, censusTag, None]
        for j in census.loc[i]:
            lrow.append(j)
        lrow.append(None)
        template.append(lrow)
        lrow = ['', '', '', None, finalTag, None]
        for j in endOfTerm.loc[i]:
            lrow.append(j)
        lrow.append(None)
        template.append(lrow)
    return template

def filter_str(row):
    newRow = []
    for j in row:
        if j != '' and j != None:
            newRow.append(j)
    return newRow

def verify(inputFile, QCData, index):
    loc = (inputFile)
    wb = xlrd.open_workbook(loc) 
    sheet = wb.sheet_by_index(index)
    QCCompleted = 1
    for i in range(sheet.nrows):
        sheetRow = filter_str(sheet.row_values(i))
        QCRow = filter_str(QCData[i])
        if sheetRow != QCRow:
            print(inputFile + " (sheet " + str(index + 1) + ") line " + str(i) + " Error!")
            print("EOY Output:")
            print(sheetRow)
            print("QC Output:")
            print(QCRow)
            QCCompleted = 0
    return True

In [52]:
def report_generator_UT_TTU(master, district):
    
    OnRamps = standardize(read_data_without_differentiate(master, district))
    UT, TTU = read_data(master, district)

    funclist = [(UT, Figure_1_4_a_QC)]
    for i in funclist:
        data, QC= i
        return verify(district + ".xlsx", QC(data, district), 4)

master = "18-19 FAKE Master File (Final).csv"
district = pd.read_csv("DistrictName2.csv", header = None)

for i in range(len(district)):
    distrct_name = district.loc[i, 0]
    try:
        if report_generator_UT_TTU(master, distrct_name) == True:
            if i < len(district) - 1:
                district.loc[i + 1 :].to_csv("district not finished.csv", index=False)
    except FileNotFoundError as error:
        print(distrct_name)
        print(error)